In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

In [2]:
data = pd.read_csv('D:/데이터/IGAWorks/reserved_encoding_data.csv')

In [8]:
df = data.copy()

In [9]:
# 일단 범주의 개수가 70만개 이상인 변수 제거하고 진행
df.drop(['device_ifa', 'install_pack', 'cate_code'] , axis=1, inplace=True)

In [33]:
cat_vars = ['ssp_id', 'campaign_id', 'adset_id', 'media_id', 'media_name', 'media_bundle',
'media_domain', 'publisher_id', 'publisher_name', 'device_os_version', 
'device_model', 'device_carrier', 'device_make',  'device_language', 'device_region', 'device_city',
 'advertisement_id']

emb_data = df[cat_vars]

In [34]:
from sklearn.preprocessing import LabelEncoder

for i in emb_data.columns:
    le = LabelEncoder()
    le.fit(emb_data[i])
    emb_data[i] = le.transform(emb_data[i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
emb_data

,ssp_id,campaign_id,adset_id,media_id,media_name,media_bundle,media_domain,publisher_id,publisher_name,device_os_version,device_model,device_carrier,device_make,device_language,device_region,device_city,advertisement_id
0,7,9,413,2825,2462,485,109,738,207,32,631,326,159,1,32,631,18
1,7,9,413,2825,2462,485,109,738,207,32,631,326,159,1,32,631,18
2,7,9,441,2825,2462,485,109,738,207,32,631,326,159,1,32,631,18
3,8,155,714,605,2519,2541,109,1021,14,14,166,213,136,6,9,275,17
4,7,9,441,2825,2462,193,109,738,207,32,631,326,159,1,32,631,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232365,13,120,90,2921,2246,863,109,2540,928,0,273,213,136,6,32,631,1
2232366,13,146,349,1308,1759,1095,109,569,453,32,631,152,159,6,32,631,4
2232367,8,78,310,2616,4474,2314,109,1041,577,14,20,213,136,6,0,912,9
2232368,7,28,491,2825,2462,485,109,738,207,32,631,326,159,1,32,631,9


In [36]:
temp = df.drop(cat_vars , axis=1)

In [39]:
temp = pd.concat([temp, emb_data] , axis=1)

In [41]:
temp.drop(['event_datetime', 'bid_id'] , axis=1, inplace=True)

In [43]:
train = temp[temp['day'].isin(list(range(1,10)))]
test = temp[temp['day'] == 10]

In [49]:
X_train = train.iloc[:, 1:]
X_test = test.iloc[:, 1:]
y_train = train['click'].values
y_test = test['click'].values

In [50]:
def get_input_data(data):
    result = list()
    for i in data.columns:
        temp = data[i].values
        result.append(temp)
    return result

In [51]:
X_train = get_input_data(X_train)
X_test = get_input_data(X_test)

In [52]:
# make output 3d
y_train = y_train.reshape((len(y_train), 1, 1))
y_test = y_test.reshape((len(y_test), 1, 1))

In [54]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.utils import plot_model

In [56]:
# prepare each input head
in_layers = list()
em_layers = list()

for i in range(len(X_train)):
    # calculate the number of unique inputs
    n_labels = len(np.unique(X_train[i]))
    # define input layer
    in_layer = Input(shape=(1,))
    # define embedding layer
    em_layer = Embedding(n_labels, 10)(in_layer)
    # store layers
    in_layers.append(in_layer)
    em_layers.append(em_layer)

Instructions for updating:
Colocations handled automatically by placer.


In [58]:
# concat all embeddings
merge = concatenate(em_layers)
dense = Dense(10, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=in_layers, outputs=output)

In [61]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

In [69]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [70]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=2)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 36 array(s), but instead got the following list of 1 arrays: [array([[8.0000e+00, 8.0000e+00, 8.0000e+00, ..., 7.0000e+00, 9.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0...